# Configuration-Based Workflows

This notebook demonstrates how to run decline curve analysis workflows using configuration files (TOML/YAML), making it easy to run analyses without writing Python code.

## Learning Objectives

By the end of this notebook, you will be able to:
- Create and use configuration files for batch DCA jobs
- Run parameter sweeps using config files
- Configure panel data analysis workflows
- Use benchmark factory with configs
- Understand the different config formats and when to use them

## Table of Contents
1. [Setup](#setup)
2. [Batch Job Configuration](#batch)
3. [Benchmark Factory Configuration](#benchmark)
4. [Panel Analysis Configuration](#panel)
5. [Creating Config Files Programmatically](#create)

## 1. Setup {#setup}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path

from decline_curve.config import (
    BatchJobConfig,
    create_example_config,
)
from decline_curve.benchmark_factory import BenchmarkFactory
from decline_curve.panel_analysis_sweep import PanelAnalysisSweep

print("✓ All imports successful!")

## 2. Batch Job Configuration {#batch}

The `BatchJobConfig` allows you to run complete DCA workflows from a single configuration file.

In [ ]:
# Create an example configuration file
config_path = "my_batch_config.toml"
create_example_config(config_path, format="toml")

print(f"✓ Example config created at: {config_path}")
print("\nConfig file contents:")
print("=" * 70)
with open(config_path, "r") as f:
    print(f.read())
print("=" * 70)

In [ ]:
# Load configuration from file
config = BatchJobConfig.from_file(config_path)

print("=" * 70)
print("LOADED CONFIGURATION")
print("=" * 70)
print(f"Data source: {config.data.path}")
print(f"Model: {config.model.model} ({config.model.kind})")
print(f"Forecast horizon: {config.model.horizon} months")
print(f"Oil price: ${config.economics.price:.2f}/bbl")
print(f"OPEX: ${config.economics.opex:.2f}/bbl")
print(f"Output directory: {config.output.output_dir}")
print(f"Parallel jobs: {config.n_jobs}")
print("=" * 70)

In [ ]:
# You can also create configs programmatically
config_dict = {
    "data": {
        "path": "data/production.csv",
        "format": "csv",
        "well_id_col": "well_id",
        "date_col": "date",
        "value_col": "oil_bbl",
    },
    "model": {
        "model": "arps",
        "kind": "hyperbolic",
        "horizon": 24,
    },
    "economics": {
        "price": 75.0,
        "opex": 18.0,
        "discount_rate": 0.12,
    },
    "output": {
        "output_dir": "output",
        "save_forecasts": True,
        "save_parameters": True,
    },
}

config_from_dict = BatchJobConfig.from_dict(config_dict)
print("✓ Config created from dictionary")
print(f"  Model: {config_from_dict.model.model}")
print(f"  Price: ${config_from_dict.economics.price:.2f}/bbl")

## 3. Benchmark Factory Configuration {#benchmark}

The `BenchmarkFactory` uses YAML configs for parameter sweeps and model comparison.

In [ ]:
# Load benchmark config
benchmark_config_path = "decline_curve/benchmark_config_example.yaml"

if Path(benchmark_config_path).exists():
    factory = BenchmarkFactory.from_config_file(benchmark_config_path)
    print("=" * 70)
    print("BENCHMARK FACTORY CONFIG")
    print("=" * 70)
    print(f"Experiment: {factory.experiment_name}")
    print(f"Models: {factory.models}")
    print(f"Horizons: {factory.horizons}")
    print(f"Parameter sweeps: {len(factory.parameter_sweeps)}")
    print("=" * 70)
else:
    print(f"Config file not found: {benchmark_config_path}")

## 4. Panel Analysis Configuration {#panel}

Panel data analysis can be configured via YAML files for regression specifications and parameter sweeps.

In [ ]:
# Load panel analysis config
panel_config_path = "decline_curve/panel_analysis_config_example.yaml"

if Path(panel_config_path).exists():
    sweep = PanelAnalysisSweep.from_config_file(panel_config_path)
    print("=" * 70)
    print("PANEL ANALYSIS CONFIG")
    print("=" * 70)
    print(f"Experiment: {sweep.config.experiment_name}")
    print(f"Data path: {sweep.config.data_path}")
    print(f"Model type: {sweep.config.model_type}")
    print(f"Parameter sweeps: {len(sweep.config.parameter_sweeps)}")
    print("=" * 70)
else:
    print(f"Config file not found: {panel_config_path}")

## 5. Creating Config Files Programmatically {#create}

You can create config files programmatically and save them for reuse.

In [ ]:
# Create a custom config
custom_config = {
    "data": {
        "path": "my_data.csv",
        "format": "csv",
        "well_id_col": "well_id",
        "date_col": "date",
        "value_col": "oil_bbl",
    },
    "model": {
        "model": "arps",
        "kind": "hyperbolic",
        "horizon": 36,
        "params": {},
    },
    "economics": {
        "price": 80.0,
        "opex": 20.0,
        "fixed_opex": 10000.0,
        "discount_rate": 0.10,
        "scenarios": [
            {"name": "low", "price": 60.0},
            {"name": "base", "price": 80.0},
            {"name": "high", "price": 100.0},
        ],
    },
    "output": {
        "output_dir": "custom_output",
        "save_forecasts": True,
        "save_parameters": True,
        "save_plots": True,
        "save_reports": True,
        "format": "csv",
    },
    "n_jobs": 4,
    "chunk_size": 50,
    "max_retries": 3,
    "log_level": "INFO",
}

# Create config object
config_obj = BatchJobConfig.from_dict(custom_config)

# Convert back to dict (useful for saving)
config_dict_saved = config_obj.to_dict()

print("✓ Custom config created and validated")
print(f"  Output directory: {config_obj.output.output_dir}")
print(f"  Save plots: {config_obj.output.save_plots}")
print(f"  Price scenarios: {len(config_obj.economics.scenarios) if config_obj.economics.scenarios else 0}")

## Summary

This notebook demonstrated:
- ✓ Creating and loading batch job configurations (TOML/YAML)
- ✓ Using configs for benchmark factory parameter sweeps
- ✓ Configuring panel data analysis workflows
- ✓ Creating configs programmatically

### Key Benefits of Config-Based Workflows

1. **No Python Required**: Edit config files instead of writing code
2. **Reproducibility**: Share config files to reproduce analyses
3. **Version Control**: Track config changes in Git
4. **Parameter Sweeps**: Easy to run multiple scenarios
5. **Documentation**: Config files serve as documentation

### Next Steps

- See `examples/config_workflow_example.py` for a complete workflow
- Check `examples/README_CLI_CONFIG.md` for CLI usage
- Review config example files in `decline_curve/` directory